In [1]:
import sys

sys.path.append('/Users/floyd/Desktop/DeepIV/deepiv')

In [2]:
from __future__ import print_function

import warnings

from models import Treatment, Response
import architectures as architectures
import densities as densities

import tensorflow as tf

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.merge import Concatenate

import numpy

import data_generator

Using TensorFlow backend.


In [3]:

n = 5000
dropout_rate = min(1000./(1000. + n), 0.5)
epochs = int(1500000./float(n)) # heuristic to select number of epochs
epochs = 300
batch_size = 100
images = False

In [4]:

def datafunction(n, s, images=images, test=False):
    return data_generator.demand(n=n, seed=s, ypcor=0.5, use_images=images, test=test)


In [5]:
datafunction(n, 1)

(array([[  4.17022005e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
        [  7.20324493e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
        [  1.14374817e-03,   1.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  3.28162589e+00,   0.00000000e+00,   1.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  9.91801159e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.50316652e+00,   0.00000000e+00,   1.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]]),
 array([[-0.16025159],
        [ 0.18191114],
        [ 0.23837009],
        ..., 
        [-0.5152147 ],
        [-0.95926628],
        [ 1.44507218]]),
 array([[ 16.61001731],
        [ 16.6

In [6]:
print('n is: ', n)
x, z, price, y, g_true = datafunction(n, 1)
l = [x, z, price, y, g_true]
for i in l:
    try:
        print(i.shape)
    except:
        print(i)

n is:  5000
(5000, 8)
(5000, 1)
(5000, 1)
(5000, 1)
<function demand.<locals>.<lambda> at 0x1a1fd78a60>


In [7]:
print(x[:3])
print(z[:3])
print(price[:3]) # here price is the treatment, in the paper, it is p.
print(y[:3])

[[  4.17022005e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  7.20324493e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  1.14374817e-03   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]]
[[-0.16025159]
 [ 0.18191114]
 [ 0.23837009]]
[[ 16.61001731]
 [ 16.68964065]
 [ 18.38537148]]
[[ -1.16091037e+01]
 [ -8.47012550e+00]
 [  8.99871909e-03]]


In [8]:
print("Data shapes:\n\
Features:{x},\n\
Instruments:{z},\n\
Treament:{price},\n\
Response:{y}".format(**{'x':x.shape, 'z':z.shape,
                        'price':price.shape, 'y':y.shape}))

Data shapes:
Features:(5000, 8),
Instruments:(5000, 1),
Treament:(5000, 1),
Response:(5000, 1)


# 2. Build and fit treatment model

In [9]:
# Build and fit treatment model
instruments = Input(shape=(z.shape[1],), name="instruments")
features    = Input(shape=(x.shape[1],), name="features")
treatment_input = Concatenate(axis=1)([instruments, features])
treatment_input

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 9) dtype=float32>

In [12]:
hidden = [128, 64, 32]

act = "relu"

n_components = 10

In [13]:
def Mix_Gaussian_Output(x):
    return densities.mixture_of_gaussian_output(x, n_components)

In [14]:
# module: architectures.py
est_treat = architectures.feed_forward_net(treatment_input, 
                                           Mix_Gaussian_Output,
                                           hidden_layers=hidden,
                                           dropout_rate=dropout_rate, 
                                           l2=0.0001,
                                           activations=act)
est_treat

<tf.Tensor 'concatenate_2/concat:0' shape=(?, 30) dtype=float32>

In [15]:
treatment_model = Treatment(inputs=[instruments, features], 
                            outputs=est_treat)
treatment_model

In [19]:
treatment_model.input

[<tf.Tensor 'instruments:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'features:0' shape=(?, 8) dtype=float32>]

In [23]:
treatment_model.output[0]

<tf.Tensor 'strided_slice_1:0' shape=(30,) dtype=float32>

In [16]:
treatment_model.compile('adam',
                        loss="mixture_of_gaussians",
                        n_components=n_components)

In [18]:
treatment_model.fit([z, x], price, epochs=epochs, batch_size=batch_size)

Epoch 1/300
5000/5000 [==============================] - 1s 286us/step - loss: 10.2379
Epoch 2/300
5000/5000 [==============================] - 0s 25us/step - loss: 5.2748
Epoch 3/300
5000/5000 [==============================] - 0s 25us/step - loss: 4.4859
Epoch 4/300
5000/5000 [==============================] - 0s 25us/step - loss: 3.6199
Epoch 5/300
5000/5000 [==============================] - 0s 25us/step - loss: 3.0833
Epoch 6/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.9071
Epoch 7/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.8507
Epoch 8/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.7916
Epoch 9/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.7782
Epoch 10/300
5000/5000 [==============================] - 0s 26us/step - loss: 2.7175
Epoch 11/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.7011
Epoch 12/300
5000/5000 [==============================] - ETA

5000/5000 [==============================] - 0s 28us/step - loss: 2.1876
Epoch 97/300
5000/5000 [==============================] - 0s 27us/step - loss: 2.1903
Epoch 98/300
5000/5000 [==============================] - 0s 26us/step - loss: 2.1796
Epoch 99/300
5000/5000 [==============================] - 0s 27us/step - loss: 2.1707
Epoch 100/300
5000/5000 [==============================] - 0s 26us/step - loss: 2.1737
Epoch 101/300
5000/5000 [==============================] - 0s 26us/step - loss: 2.1740
Epoch 102/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.1685
Epoch 103/300
5000/5000 [==============================] - 0s 26us/step - loss: 2.1834
Epoch 104/300
5000/5000 [==============================] - 0s 27us/step - loss: 2.1622
Epoch 105/300
5000/5000 [==============================] - 0s 27us/step - loss: 2.1575
Epoch 106/300
5000/5000 [==============================] - 0s 25us/step - loss: 2.1612
Epoch 107/300
5000/5000 [==============================] - 0

5000/5000 [==============================] - 0s 26us/step - loss: 1.8262
Epoch 285/300
5000/5000 [==============================] - 0s 24us/step - loss: 1.8568
Epoch 286/300
5000/5000 [==============================] - 0s 26us/step - loss: 1.8232
Epoch 287/300
5000/5000 [==============================] - 0s 26us/step - loss: 1.8383
Epoch 288/300
5000/5000 [==============================] - 0s 26us/step - loss: 1.8480
Epoch 289/300
5000/5000 [==============================] - 0s 26us/step - loss: 1.8438
Epoch 290/300
5000/5000 [==============================] - 0s 25us/step - loss: 1.8225
Epoch 291/300
5000/5000 [==============================] - 0s 24us/step - loss: 1.8291
Epoch 292/300
5000/5000 [==============================] - 0s 26us/step - loss: 1.8180
Epoch 293/300
5000/5000 [==============================] - 0s 25us/step - loss: 1.8403
Epoch 294/300
5000/5000 [==============================] - 0s 25us/step - loss: 1.8256
Epoch 295/300
5000/5000 [==============================] 

In [38]:
z_tr = z[:3]
z_tr

array([[-0.16025159],
       [ 0.18191114],
       [ 0.23837009]])

In [40]:
z_tr = z[:3]
z_tr.repeat(5, axis = 0)

array([[-0.16025159],
       [-0.16025159],
       [-0.16025159],
       [-0.16025159],
       [-0.16025159],
       [ 0.18191114],
       [ 0.18191114],
       [ 0.18191114],
       [ 0.18191114],
       [ 0.18191114],
       [ 0.23837009],
       [ 0.23837009],
       [ 0.23837009],
       [ 0.23837009],
       [ 0.23837009]])

In [42]:
x_tr = x[:3]
x_tr

array([[  4.17022005e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  7.20324493e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  1.14374817e-03,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00]])

In [41]:
# Mixtured Gaussian Distribution
treatment_model.sample([z[:3], x[:3]], n_samples= 4)

array([[ 17.8463974 ],
       [ 18.52820396],
       [ 14.04855633],
       [ 14.91806698],
       [ 19.63589096],
       [ 17.49326706],
       [ 18.55532455],
       [ 16.04458618],
       [ 20.13111496],
       [ 20.06375313],
       [ 17.62745285],
       [ 21.5983963 ]], dtype=float32)

# 3. Build Response Model

In [29]:
# Build and fit response model
# not input p directly
treatment = Input(shape=(t.shape[1],), name="treatment") # price in the paper
response_input = Concatenate(axis=1)([features, treatment]) # p and x

In [44]:
print(hidden)
print(act)

[128, 64, 32]
relu


In [30]:
est_response = architectures.feed_forward_net(response_input, 
                                              Dense(1),
                                              activations=act,
                                              hidden_layers=hidden,
                                              l2=0.001,
                                              dropout_rate=dropout_rate)
est_response

<tf.Tensor 'dense_7/BiasAdd:0' shape=(?, 1) dtype=float32>

In [31]:
response_model = Response(treatment=treatment_model,
                          inputs=[features, treatment],
                          outputs=est_response)
response_model

In [45]:
epochs

300

In [32]:
response_model.compile('adam', 
                       loss='mse')
# [z, x], this first show always be z, the instrumental variable
response_model.fit([z, x], y, 
                   epochs=epochs, 
                   verbose=1,
                   batch_size=batch_size, 
                   samples_per_batch=2)


Epoch 1/300
50/50 [==============================] - 1s 12ms/step - loss: 1.9729
Epoch 2/300
50/50 [==============================] - 0s 4ms/step - loss: 1.4911
Epoch 3/300
50/50 [==============================] - 0s 4ms/step - loss: 1.3378
Epoch 4/300
50/50 [==============================] - 0s 4ms/step - loss: 1.3288
Epoch 5/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2901
Epoch 6/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2708A: 0s - loss: 1.26
Epoch 7/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2538
Epoch 8/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2649
Epoch 9/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2702
Epoch 10/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2692
Epoch 11/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2463
Epoch 12/300
50/50 [==============================] - 0s 4ms/step - loss: 1.2370
Epoch 13/300
50/50

50/50 [==============================] - 0s 4ms/step - loss: 1.0326
Epoch 199/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0497
Epoch 200/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0491
Epoch 201/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0512
Epoch 202/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0391
Epoch 203/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0478
Epoch 204/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0410
Epoch 205/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0493
Epoch 206/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0413A: 0s - loss: 1.026
Epoch 207/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0441
Epoch 208/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0583
Epoch 209/300
50/50 [==============================] - 0s 4ms/step - loss: 1.0254
Epoch 210/3

In [33]:
oos_perf = data_generator.monte_carlo_error(lambda x,z,t: response_model.predict([x,t]), 
                                            datafunction, 
                                            has_latent=images, 
                                            debug=False)


print("Out of sample performance evaluated against the true function: %f" % oos_perf)

Out of sample performance evaluated against the true function: 0.088323
